# Preprocessing Household Survey Data Of Tanzania

In this Python notebook, we will explore the process of preprocessing household survey data from Tanzania to compute two food security indicators: **Food Consumption Score (FCS)** and **Household Dietary Diversity Score (HDDS)**. These indicators play a crucial role in assessing food security and nutritional status at the household level. The aim of calculating these indicators is to use them as ground truth data in the research project of using machine learning and deep learning for prediction for security indicators(FCS and HDDS) from heterogenous data.


### Food Security Indicators
    
These are quantitative measures used to evaluate the accessibility, availability, and utilization of food at various levels starting from households,community to the national level. These indicators provide insights into the extent and severity of food insecurity, helping policymakers, researchers, and practitioners to identify vulnerable populations and design targeted interventions. There are numbers of food security indicators however as we mentioned early we will only focus on two indicators which are **Food Consumption Score(FCS)** and **Household Dietary Diversity Score(HDDS)**. The details description of these indicators will be given to the [Computation of FCS and HDDS](#compute_fcs_hdds) section.
    


### About Tanzania
    
Tanzania is a country located in East Africa,it's economy is primarily driven by agriculture,despite being an agricultural country, Tanzania faces challenges with malnutrition and lack of dietary diversity, particularly in rural areas.Many households rely heavily on staple crops such as maize, rice, and cassava, leading to a limited variety in diets and micronutrient deficiencies. Also poverty remains a significant barrier that makes many households struggle to afford an adequate and nutritious diet. Apart from that limited access to markets, infrastructure, and transportation exacerbates food insecurity, particularly in remote and rural areas.
    

### Data Source

We will be working with National Panel Survey (NPS) data from Tanzania which are public available on [National Bureau of Statistics (NBS)](https://www.nbs.go.tz/index.php/en/)
. The NPS is a household survey conducted at a national level, offering insights into poverty levels, agricultural productivity, and various crucial development metrics. It's characterized as an "integrated" survey because it encompasses diverse subjects within a single questionnaire, spanning from education and healthcare to issues like crime, gender-based violence and food security. Since our topic is concering about food security we will only focus on the data which are relevant to our topic.

#### The following are the datasets that we will work with:

* [National Panel Survey(NPS) 2008 - 2009](https://microdata.worldbank.org/index.php/catalog/76)

* [2009 - Comprehensive Food Security and Vulnerability Analysis (CFSVA)](https://microdata.statistics.gov.rw/index.php/catalog/8)

* [2012 - Comprehensive Food Security and Vulnerability Analysis (CFSVA)](https://microdata.statistics.gov.rw/index.php/catalog/69)

* [2015 - Comprehensive Food Security and Vulnerability Analysis (CFSVA)](https://microdata.statistics.gov.rw/index.php/catalog/70)

* [2018 - Comprehensive Food Security and Vulnerability Analysis (CFSVA)](https://microdata.statistics.gov.rw/index.php/catalog/91)

* [2021 - Comprehensive Food Security and Vulnerability Analysis (CFSVA)](https://microdata.statistics.gov.rw/index.php/catalog/106)


## Import Libraries

In this section, we import essential libraries and modules required for data preprocessing, analysis, and visualization tasks. These libraries provide robust functionalities and tools that streamline the data analysis workflow and enable us to manipulate and explore the dataset efficiently.

In [14]:
#uncomment to install savReaderWriter
#!pip install savReaderWriter

In [15]:
import pandas as pd
import pyreadstat as ps
from pandas import read_csv
import savReaderWriter as sv

## <a id='helper_function'></a> Helper Function

In this section we define a set of helper functions designed to streamline data preprocessing tasks and facilitate the computation of Food Consumption Score (FCS) and Household Dietary Diversity Score (HDDS). These functions are designed to assist in converting dataset files from different formats to a common format (i.e., .csv), making them compatible with various data analysis tools and workflows.

In [16]:
'''
    This function decodes bytes to string and handles integer values.
    It checks if the input value is a bytes object and decodes it to a UTF-8 encoded string.
    If the value is a float and represents an integer, it converts it to an integer.
    Otherwise, it returns the original value.
    It is important while converting the .sav file to .csv.
'''

def decode_value(value):
    if isinstance(value, bytes):
        return value.decode('utf-8')
    elif isinstance(value, float) and value.is_integer():
        return int(value)
    else:
        return value

In [17]:
'''
    This function converts a .sav (SPSS) file to a .csv (comma-separated values) file.
    It reads the .sav file using sv.SavReader, extracts column names, and decodes values using the decode_value function.
    The data is then converted to a DataFrame and saved as a .csv file at the specified path.
'''

def sav_to_csv(sav_path,csv_path):
    with sv.SavReader(sav_path) as reader:
        # Extract the column names
        column_names = [name.decode('utf-8') for name in reader.header]
        
        # Read the data and decode values
        data = [[decode_value(value) for value in row] for row in reader]
    
    df = pd.DataFrame(data, columns=column_names)
    df.to_csv(csv_path, index=False)
    print(f"A new dataset is saved to {csv_path}")

In [18]:
'''
    This function converts a .dta (Stata) file to a .csv file.
    It reads the .dta file using pd.read_stata and loads it into a DataFrame.
    The DataFrame is then saved as a .csv file at the specified path
'''

def dta_to_csv(dta_path, csv_path):
    df = pd.read_stata(dta_path)
    df.to_csv(csv_path, index=False)
    print(f"A new dataset is saved to {csv_path}")

In [19]:
'''
    This function is designed to subset a DataFrame based on specified columns, add a year column
    and save the resulting subset to a new CSV file. 
    This function is helful when your dataset have a large number of columns and you only need to work with some of them.
    
    Parameters:
        df (DataFrame): The original DataFrame.
        columns_to_keep (list): A list of column names to keep.
        output_file (str): The path to the output CSV file.
        year(int): The year associated with the dataset, which will be added as a new column.
        rename_columns (dict, optional): A dictionary where keys are original column names and values are new names.
'''

def subset_and_save(df, columns_to_keep, output_file, year, rename_columns=None):
    
    # Selecting columns to keep
    df_subset = df[columns_to_keep]

    # Optionally renaming columns
    if rename_columns:
        df_subset = df_subset.rename(columns=rename_columns)
    
    # Adding a year column
    df_subset.insert(0, 'year', year)

    # Save the subset DataFrame to a new CSV file
    df_subset.to_csv(output_file, index=False)

    print(f"A new dataset is saved to {output_file}")

## <a id='compute_fcs_hdds'></a> Computation of FCS and HDDS

In this section, we will explain how we will compute the **Food Consumption Score (FCS)** and the **Household Dietary Diversity Score (HDDS)**. Since we will work of multiple dataset in our study area, we will adopt a standardized approach for calculating both FCS and HDDS across all datasets. Regardless of the dataset's specific characteristics, we will apply consistent methods to ensure comparability and reliability of the results.

###  <a id='fcs'></a>1. Food Consumption Score (FCS)

The Food Consumption Score (FCS) is a food security indicator developed by the [World Food Programme (WFP)](https://resources.vam.wfp.org/data-analysis/quantitative/food-security/food-consumption-score). It serves as an essential index for assessing household food consumption patterns and nutritional adequacy. The data to compute the FCS are collected by household survey questionnaire by asking a respondent the list of food groups they have consumed for the past seven days. The FCS aggregates these data on the diversity and frequency of food groups consumed over the previous seven days **(7)** which is then weighted according to the relative nutritional value of the consumed food groups as specified in the table below.

<p><img src="images/food_weights.png"  align="centre" alt="food groups weighs" style="width:600px;height:300px;"></p>


#### Steps to Compute FCS
 1. Group food items in the specified food groups
 2. Sum all the consumption frequencies of food items within the same group but the maximum limit should be 7
 3. Multiply the value of each food group by its weight as explained in table
 4. Sum the weighted food group scores to obtain the overall FCS
 5. Determine the household's food consumption status based on the following thresholds: 
     * 0 - 21 : Poor
     * 21.5 - 35 : Borderline
     * &gt; 35 : Acceptable
     
##### Mathematically the FCS is represented as:

$$
FCS = \sum_{j=1}^{9}f_j \times x_j
$$

**Where:**

- $(f_j$) represents the frequency of consumption of food group $(j$).
- $(x_j$) represents the nutritional value (weight) of food group $(j$).
- $(j$) ranges from 1 to 9, representing the nine food groups considered in the calculation.

For more detailed description about FCS Calculation and uses in food security analysis can be found in [this Document](https://documents.wfp.org/stellent/groups/public/documents/manual_guide_proced/wfp197216.pdf). The source code and sample of the data can also be found [Here](https://resources.vam.wfp.org/data-analysis/quantitative/food-security/food-consumption-score)

#### Sum Up Frequency by Food Group  Function (SFFGF)

This function implement the **Step 1** and **Step 2** as explained above.
It Groups all the food items to its corresponding food groups and compute the sum of all the consumption frequencies. The upper limit of the food frequency of each food groups is set to a maximum of 7 to prevent biasing the score upwards. This approach is used to prevent the bias of the score when there is an increasing of the number of food items/groups used in questionnaire(and later collapsed into the food groups for the FCS calculation). Assuming the questionnaire gathers the consumption of maize, rice,wheat and these foods are all consumed in combinatination, let say the frequency of maize=3, rice=3, wheat=3, all of these fall into the food group of cereals then the frequency of the cereals will be 9, thus biasing the score. That is the reason why we set the maximum frequency to 7.

In [79]:
def sum_up_frequency_by_food_group(data):
    
    '''
    Processes the given DataFrame by grouping by 'hhid' and 'food_group',
    summing up 'is_consumed', pivoting the data to get food groups as columns, and
    ensuring that no aggregated sum exceeds 7.

    Parameters:
    - data (pd.DataFrame): The input DataFrame with at least 'hhid','food_group', and 'is_consumed' columns.
    - hhid: represent a unique identifier of each household
    - food_groups: containing the food group such as cereal,pulses,sugar, oil, etc
    - is_consumed: represent an integer value of either 1(if it is consumed) or 0 (if not consumed)

    Returns:
    - pd.DataFrame: A pivoted DataFrame with 'hhid' as rows, 'food_group' as columns,
                    and sum of 'is_consumed' as values, capped at a maximum of 7.
                    
    '''
    # Group by 'hhid', 'food_group',then sum up 'is_consumed'
    grouped_data = data.groupby(['hhid', 'food_group'])['is_consumed'].sum().reset_index()

    # Pivot the data to get food groups as columns
    pivot_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)

    # Reset index to make 'hhid' a column again
    pivot_data.reset_index(inplace=True)

    # Set the maximum number of days to be 7 for all columns except 'hhid'
    pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].applymap(lambda x: min(x, 7))

    return pivot_data

#### Calculate Food Consumption Score Function (CFCSF)

This function compute the food consumption score of a given dataframe. It implements the instruction of **Step 1** to **Step 4**  to compute the overall score. The weights of each food group is defined as explained in the section above. There are two version of this function, **calculate_fcs_version1()** only return the hhid and the computed food consumption score(fcs) and the other one is **calculate_fcs_version2()** which return the hhid, the columns of each food groups with their aggregated frequency and the calculated the food consumption score. Both of these methods produce the same score for the fcs but diffrent is only on the returned dataframe. So the choice of which one you want to use depends with your goals.

In [72]:
def calculate_fcs_version1(data):
    '''
    Processes the given DataFrame by grouping by 'hhid', 'food_group', and 
    summing up 'is_consumed', pivoting the data to get food groups as columns, capping
    the values at a maximum of 7, assigning weights to each food group, and finally
    calculating the food consumption score for each household.

    Parameters:
    - data (pd.DataFrame): The input DataFrame with at least 'hhid', 'food_group' and 'is_consumed' columns.
    - hhid: represent a unique identifier of each household
    - food_groups: containing the food group such as cereal,pulses,sugar, oil, etc
    - is_consumed: represent an integer value of either 1(if it is consumed) or 0 (if not consumed)

    Returns:
    - pd.DataFrame: A DataFrame with 'hhid' and 'food_consumption_score' columns,
                    representing the calculated food consumption score for each household.
    '''
    # Define weights for each food group
    weights = {
        'cereals_tubers': 2,
        'pulses_nuts': 3,
        'vegetables_leaves': 1,
        'fruits': 1,
        'animal_protein': 4,
        'dairy_products': 4,
        'sugar': 0.5,
        'oil': 0.5,
        'condiments': 0
    }
    
    # Perform initial processing and capping
    grouped_data = data.groupby(['hhid', 'food_group'])['is_consumed'].sum().reset_index()
    pivot_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)
    pivot_data.reset_index(inplace=True)
    pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].applymap(lambda x: min(x, 7))

    # Calculate the food consumption score
    for column in pivot_data.columns[1:]:
        if column in weights:
            pivot_data[column] = pivot_data[column] * weights[column]
        else:
            print(f"Warning: '{column}' not found in weights; it will be ignored in score calculation.")

    pivot_data['food_consumption_score'] = pivot_data.iloc[:, 1:].sum(axis=1)

    # Return a DataFrame with 'hhid' and 'food_consumption_score'
    return pivot_data[['hhid', 'food_consumption_score']]

In [73]:
def calculate_fcs_version2(data):
    
    ''''
    Processes the given DataFrame by grouping by 'hhid', 'food_group', and 
    summing up 'is_consumed', pivoting the data to get food groups as columns, capping
    the values at a maximum of 7, assigning weights to each food group, and finally
    calculating the food consumption score for each household.

    Parameters:
    - data (pd.DataFrame): Input DataFrame with 'hhid', 'food_group', 'food_items', and 'is_consumed'.
    - hhid: represent a unique identifier of each household
    - food_items: represent the food items such as maize, rice etc
    - food_groups: containing the food group such as cereal,pulses,sugar, oil, etc
    - is_consumed: represent an integer value of either 1(if it is consumed) or 0 (if not consumed)
    
    Returns:
    - pd.DataFrame: Output DataFrame with each 'hhid', the aggregated 'is_consumed' values for each food group,
                    and the calculated FCS.
    '''
    # Define weights for each food group
    weights = {
        'cereals_tubers': 2,
        'pulses_nuts': 3,
        'vegetables_leaves': 1,
        'fruits': 1,
        'animal_protein': 4,
        'dairy_products': 4,
        'sugar': 0.5,
        'oil': 0.5,
        'condiments': 0
    }
    
    # Group by 'hhid' and 'food_group', then sum 'is_consumed', capping at 7
    grouped = data.groupby(['hhid', 'food_group',])['is_consumed'].sum().reset_index()
    grouped['is_consumed'] = grouped['is_consumed'].apply(lambda x: min(x, 7))
    
    # Pivot the data to get food groups as columns, filled with 'is_consumed' values
    pivot_data = grouped.pivot(index='hhid', columns='food_group', values='is_consumed').fillna(0)
    
    # Compute the FCS by applying weights to the 'is_consumed' values and summing them up
    fcs = pivot_data.copy()
    for food_group in weights:
        if food_group in fcs.columns:
            fcs[food_group] = fcs[food_group] * weights[food_group]
    
    pivot_data['fcs'] = fcs.sum(axis=1)
    
    # Reset index to convert 'hhid' from index to a column
    pivot_data.reset_index(inplace=True)
    
    return pivot_data

### 2. Household Dietary Diversity Score (HDDS)

Household dietary diversity Score (HDDS) is a qualitative measure of food consumption that reflects household access to a variety of foods that indicate dietary diversity and nutritional quality.The HDDS consists of a simple count of food groups that a household has consumed over the preceding 24 hours. Each food group is assigned a score of **1 (if consumed over the previous 24 hours)** or **0 (if not consumed in the last 24 hours)**. The household score will range between 0 to 12 and is equal to the total number of food groups consumed by the household.

##### The following 12 food groups are used to calculate the HDDS indicator:

<img src="images/hdds_groups.png"  align="centre" alt="food groups weighs" style="width:400px;height:200px;">



##### Mathematically the HDDS is represented as: 
$$
HDDS = \sum_{j=1}^{12} x_j
$$

where $( x_j $) equals 1 if the household consumed food from group $( j $) in the past 24 hours, and 0 otherwise.



#### Calculate Household Dietary Diversity Score Function (CHDDSF)
This function compute the household dietary diversity score for a given data frame. 

## National Panel Survey 2008 - 2009

This dataset is public available on [National Panel Survey(NPS) 2008 - 2009](https://microdata.worldbank.org/index.php/catalog/76). The NPS interviewed 3,280 households spanning all regions and all districts of Tanzania, both mainland and Zanzibar.The dataset containing data related to Household questionnaire, Agriculture questionnaire and Community questionnaire which cover broad range of the topics. For more discription about the coverage of the topics in this dataset [Click Here](https://microdata.worldbank.org/index.php/catalog/76/study-description). The dataset consinst of multiples files, however for our task we will mostly focus on HouseHold Questionnaire, for more description about the data definition of each files and their associate variables [Visit Here](https://microdata.worldbank.org/index.php/catalog/76/data-dictionary)

#### Loading the data sources

The data files we have use in the dataset are those named:
* SEC_A_T.dta : Contain the description about the address of the household which is identified by a unique identification (hhid)
* SEC_K1.dta : Contain the data information about food consumption of 7 days recall

These files are in stata format, therefore we need to change them into csv format for easily manipulation

In [12]:
#convert SEC_A_T.dta to csv file

o_path= 'Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_A_T.dta'
u_path = 'Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_A_T.csv'

dta_to_csv(o_path,u_path) #call the function to convert

A new dataset is saved to Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_A_T.csv


In [13]:
#convert SEC_K1.dta to csv file

o_path= 'Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_K1.dta'
u_path = 'Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_K1.csv'

dta_to_csv(o_path,u_path) #call the function to convert

A new dataset is saved to Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/SEC_K1.csv


Renaming of the columns and removing some of them is done on OpenRefine software but it can also be done here by using the function **subset_and_save** in the Helper Function Section. The dataset 

In [74]:
food_consumed =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_food_consumed.csv', header=0,delimiter=',')
food_consumed.head()

,hhid,food_items,is_consumed
0,1010140020171,Rice (paddy),0
1,1010140020171,Rice (husked),0
2,1010140020171,"Maize (green, cob)",0
3,1010140020171,Maize (grain),0
4,1010140020171,Maize (flour),0


In [58]:
food_consumed.shape

(192635, 3)

In [75]:
food_group =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_food_groups_for_fcs.csv', header=0,delimiter=',')
food_group.head()

,food_code,food_items,food_group
0,101,Rice (paddy),cereals_tubers
1,102,Rice (husked),cereals_tubers
2,103,"Maize (green, cob)",cereals_tubers
3,104,Maize (grain),cereals_tubers
4,105,Maize (flour),cereals_tubers


In [76]:
food_group.shape

(59, 3)

In [77]:
#merge the food consumed and food group
data = pd.merge(food_consumed, food_group, on='food_items', how='left')
data.head()

,hhid,food_items,is_consumed,food_code,food_group
0,1010140020171,Rice (paddy),0,101,cereals_tubers
1,1010140020171,Rice (husked),0,102,cereals_tubers
2,1010140020171,"Maize (green, cob)",0,103,cereals_tubers
3,1010140020171,Maize (grain),0,104,cereals_tubers
4,1010140020171,Maize (flour),0,105,cereals_tubers


In [78]:
new_data=sum_up_frequency_by_food_group(data)
new_data.head()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
0,1010140020171,2,1,2,1,0,1,0,1,2
1,1010140020284,0,1,1,0,0,1,0,0,2
2,1010140020297,3,2,2,1,0,1,1,1,2
3,1010140020409,1,2,2,0,0,1,0,1,2
4,1010140020471,0,3,2,0,0,1,1,1,2


In [80]:
new_data=sum_up_frequency_by_food_group(data)
new_data.head()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
0,1010140020171,2,1,2,1,0,1,0,1,2
1,1010140020284,0,1,1,0,0,1,0,0,2
2,1010140020297,3,2,2,1,0,1,1,1,2
3,1010140020409,1,2,2,0,0,1,0,1,2
4,1010140020471,0,3,2,0,0,1,1,1,2


In [322]:
#save to new data to csv might be useful later
data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_0.csv', index=False)
data.shape

(192635, 5)

### Calculating Food Consumption Score (FCS)

In [20]:
def sum_by_food_group(data):
    """
    Processes the given DataFrame by grouping by 'hhid', 'food_group', and 'food_items',
    summing up 'is_consumed', pivoting the data to get food groups as columns, and
    ensuring that no aggregated sum exceeds 7.

    Parameters:
    - data (pd.DataFrame): The input DataFrame with at least 'hhid', 'food_group',
      'food_items', and 'is_consumed' columns.

    Returns:
    - pd.DataFrame: A pivoted DataFrame with 'hhid' as rows, 'food_group' as columns,
                    and sum of 'is_consumed' as values, capped at a maximum of 7.
    """
    # Group by 'hhid', 'food_group', and 'food_items', then sum up 'is_consumed'
    grouped_data = data.groupby(['hhid', 'food_group', 'food_items'])['is_consumed'].sum().reset_index()

    # Pivot the data to get food groups as columns
    pivot_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)

    # Reset index to make 'hhid' a column again
    pivot_data.reset_index(inplace=True)

    # Set the maximum number of days to be 7 for all columns except 'hhid'
    pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].applymap(lambda x: min(x, 7))

    return pivot_data

In [29]:
def calculate_food_consumption_score(data):
    """
    Processes the given DataFrame by grouping by 'hhid', 'food_group', and 'food_items',
    summing up 'is_consumed', pivoting the data to get food groups as columns, capping
    the values at a maximum of 7, assigning weights to each food group, and finally
    calculating the food consumption score for each household.

    Parameters:
    - data (pd.DataFrame): The input DataFrame with at least 'hhid', 'food_group',
      'food_items', and 'is_consumed' columns.

    Returns:
    - pd.DataFrame: A DataFrame with 'hhid' and 'food_consumption_score' columns,
                    representing the calculated food consumption score for each household.
    """
    # Define weights for each food group
    weights = {
        'cereals_tubers': 2,
        'pulses_nuts': 3,
        'vegetables_leaves': 1,
        'fruits': 1,
        'animal_protein': 4,
        'dairy_products': 4,
        'sugar': 0.5,
        'oil': 0.5,
        'condiments': 0
    }
# Perform initial processing and capping
    grouped_data = data.groupby(['hhid', 'food_group'])['is_consumed'].sum().reset_index()
    pivot_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)
    pivot_data.reset_index(inplace=True)
    pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].applymap(lambda x: min(x, 7))

    # Calculate the food consumption score
    food_groups_involved = []
    for column in pivot_data.columns[1:]:
        if column in weights:
            pivot_data[column] = pivot_data[column] * weights[column]
            food_groups_involved.append(column)
        else:
            print(f"Warning: '{column}' not found in weights; it will be ignored in score calculation.")

    pivot_data['food_consumption_score'] = pivot_data.iloc[:, 1:].sum(axis=1)

    # Since every hhid now has the same set of food_groups after the pivot, we simplify by listing them all
    pivot_data['food_groups'] = ', '.join(food_groups_involved)

    # Return a DataFrame with 'hhid', 'food_consumption_score', and 'food_groups'
    return pivot_data[['hhid', 'food_consumption_score', 'food_groups']]

In [68]:
data= read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_0.csv',header=0,delimiter=',')
data.head()

,hhid,food_items,is_consumed,food_code,food_group
0,1010140020171,Rice (paddy),0,101,cereals_tubers
1,1010140020171,Rice (husked),0,102,cereals_tubers
2,1010140020171,"Maize (green, cob)",0,103,cereals_tubers
3,1010140020171,Maize (grain),0,104,cereals_tubers
4,1010140020171,Maize (flour),0,105,cereals_tubers


In [70]:
new_data = calculate_fcs_without_weighting_columns(data)
new_data.tail()
#new_data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_fcs_new.csv', index=False)

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves,fcs
3260,55020180210054,3,6,4,1,2,1,2,1,2,39.0
3261,55020180210058,1,5,2,0,0,0,1,1,2,19.5
3262,55020180210059,1,5,2,0,1,0,1,1,1,19.5
3263,55020180210068,1,5,2,0,0,0,2,2,2,23.0
3264,55020180210078,1,2,2,0,0,0,1,1,2,13.5


In [37]:
#compare the results obtained
data1= read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_fcs.csv',header=0,delimiter=',')
data1.tail()

,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves,fcs
3260,55020180210054,3,6,4,1,2,1,2,1,2,39.0
3261,55020180210058,1,5,2,0,0,0,1,1,2,19.5
3262,55020180210059,1,5,2,0,1,0,1,1,1,19.5
3263,55020180210068,1,5,2,0,0,0,2,2,2,23.0
3264,55020180210078,1,2,2,0,0,0,1,1,2,13.5


In [38]:
data2= read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_fcs_new.csv',header=0,delimiter=',')
data2.tail()

,hhid,food_consumption_score,food_groups
3260,55020180210054,39.0,"animal_protein, cereals_tubers, condiments, da..."
3261,55020180210058,19.5,"animal_protein, cereals_tubers, condiments, da..."
3262,55020180210059,19.5,"animal_protein, cereals_tubers, condiments, da..."
3263,55020180210068,23.0,"animal_protein, cereals_tubers, condiments, da..."
3264,55020180210078,13.5,"animal_protein, cereals_tubers, condiments, da..."


In [25]:
new_data.tail()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
3260,55020180210054,3,6,4,1,2,1,2,1,2
3261,55020180210058,1,5,2,0,0,0,1,1,2
3262,55020180210059,1,5,2,0,1,0,1,1,1
3263,55020180210068,1,5,2,0,0,0,2,2,2
3264,55020180210078,1,2,2,0,0,0,1,1,2


In [26]:
# Group by hhid, food_group, and food_items, then sum up is_consumed

grouped_data = data.groupby(['hhid', 'food_group', 'food_items'])['is_consumed'].sum().reset_index()

# Pivot the data to get food groups as columns
pivot_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)

# Reset index to make hhid a column again
pivot_data.reset_index(inplace=True)

pivot_data.tail()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
3260,55020180210054,3,6,4,1,2,1,2,1,2
3261,55020180210058,1,5,2,0,0,0,1,1,2
3262,55020180210059,1,5,2,0,1,0,1,1,1
3263,55020180210068,1,5,2,0,0,0,2,2,2
3264,55020180210078,1,2,2,0,0,0,1,1,2


In [357]:
pivot_data.describe()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
count,3.265000e+03,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000
mean,1.654744e+13,1.581623,4.060949,2.064012,0.361409,0.821746,0.788668,1.438591,0.888515,1.706891
std,1.599970e+13,1.205647,2.075982,0.947717,0.577758,0.942309,0.504933,0.808188,0.646308,0.676210
min,1.010140e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.010212e+12,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
50%,1.101002e+13,1.000000,4.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000
75%,1.802031e+13,2.000000,5.000000,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000
max,5.502018e+13,6.000000,15.000000,6.000000,3.000000,4.000000,2.000000,4.000000,3.000000,3.000000


In [358]:
# Set the maximum number of days to be 7.
pivot_data.iloc[:, 1:] = pivot_data.iloc[:, 1:].applymap(lambda x: 7 if x > 7 else x)
pivot_data.describe()

food_group,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
count,3.265000e+03,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000
mean,1.654744e+13,1.581623,3.959877,2.064012,0.361409,0.821746,0.788668,1.438591,0.888515,1.706891
std,1.599970e+13,1.205647,1.861779,0.947717,0.577758,0.942309,0.504933,0.808188,0.646308,0.676210
min,1.010140e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.010212e+12,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
50%,1.101002e+13,1.000000,4.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000
75%,1.802031e+13,2.000000,5.000000,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000
max,5.502018e+13,6.000000,7.000000,6.000000,3.000000,4.000000,2.000000,4.000000,3.000000,3.000000


In [359]:
#save the file
pivot_data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_1.csv', index=False)

##### Computing FCS

In [360]:
data= read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_1.csv',header=0,delimiter=',')
data.head()

,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves
0,1010140020171,2,1,2,1,0,1,0,1,2
1,1010140020284,0,1,1,0,0,1,0,0,2
2,1010140020297,3,2,2,1,0,1,1,1,2
3,1010140020409,1,2,2,0,0,1,0,1,2
4,1010140020471,0,3,2,0,0,1,1,1,2


In [362]:
#calculate the FCS based on the weight of WFP
data['fcs'] = (
    data['cereals_tubers'] * 2 +
    data['pulses_nuts'] * 3 +
    data['vegetables_leaves'] * 1 +
    data['fruits'] * 1 +
    data['animal_protein'] * 4 +
    data['dairy_products'] * 4 +
    data['sugar'] * 0.5 +
    data['oil'] * 0.5 +
    data['condiments'] * 0
)
data.head()

,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves,fcs
0,1010140020171,2,1,2,1,0,1,0,1,2,17.0
1,1010140020284,0,1,1,0,0,1,0,0,2,4.5
2,1010140020297,3,2,2,1,0,1,1,1,2,26.0
3,1010140020409,1,2,2,0,0,1,0,1,2,11.0
4,1010140020471,0,3,2,0,0,1,1,1,2,12.0


In [364]:
#save the data to csv
data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_fcs.csv', index=False)

### Calculating Household Dietry Diversity Score (HDDS)

In [365]:
#loading the files containing the food items consumed by household
food_consumed =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_food_consumed.csv', header=0,delimiter=',')
food_consumed.head()

,hhid,food_items,is_consumed
0,1010140020171,Rice (paddy),0
1,1010140020171,Rice (husked),0
2,1010140020171,"Maize (green, cob)",0
3,1010140020171,Maize (grain),0
4,1010140020171,Maize (flour),0


In [366]:
#load the file containing the food groups for calculating HDDS
food_group =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_food_groups_for_hdds.csv', header=0,delimiter=',')
food_group.head()

,food_code,food_items,food_group
0,101,Rice (paddy),cereals
1,102,Rice (husked),cereals
2,103,"Maize (green, cob)",cereals
3,104,Maize (grain),cereals
4,105,Maize (flour),cereals


In [367]:
#merge the food consumed and food group
data = pd.merge(food_consumed, food_group, on='food_items', how='left')
data.head()

,hhid,food_items,is_consumed,food_code,food_group
0,1010140020171,Rice (paddy),0,101,cereals
1,1010140020171,Rice (husked),0,102,cereals
2,1010140020171,"Maize (green, cob)",0,103,cereals
3,1010140020171,Maize (grain),0,104,cereals
4,1010140020171,Maize (flour),0,105,cereals


In [368]:
# Group by hhid, food_group, and food_items, then sum up is_consumed
grouped_data = data.groupby(['hhid', 'food_group', 'food_items'])['is_consumed'].sum().reset_index()

# Pivot the data to get food groups as columns
new_data = grouped_data.pivot_table(index='hhid', columns='food_group', values='is_consumed', aggfunc='sum', fill_value=0)

# Reset index to make hhid a column again
new_data.reset_index(inplace=True)

new_data.head()


food_group,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables
0,1010140020171,1,2,0,1,0,1,1,1,0,0,1,2
1,1010140020284,1,1,0,0,0,0,0,1,0,0,0,2
2,1010140020297,2,2,0,0,0,3,1,1,1,0,1,2
3,1010140020409,2,2,0,0,0,1,0,1,0,0,1,2
4,1010140020471,3,2,0,0,0,0,0,1,1,0,1,2


In [369]:
new_data.describe()

food_group,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables
count,3.265000e+03,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000
mean,1.654744e+13,2.576417,2.064012,0.185605,0.731087,0.821746,0.664931,0.361409,0.788668,1.438591,1.484533,0.888515,1.706891
std,1.599970e+13,1.428138,0.947717,0.388847,0.605301,0.942309,0.786185,0.577758,0.504933,0.808188,1.222425,0.646308,0.676210
min,1.010140e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.010212e+12,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
50%,1.101002e+13,3.000000,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000
75%,1.802031e+13,3.000000,3.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000
max,5.502018e+13,9.000000,6.000000,1.000000,2.000000,4.000000,4.000000,3.000000,2.000000,4.000000,6.000000,3.000000,3.000000


In [370]:
# If the sum of each food group is greater than 0, set the value to 1 so that we can compute the HDDS easily
new_data.iloc[:, 1:] = new_data.iloc[:, 1:].applymap(lambda x: 1 if x > 0 else 0)
new_data.head()

food_group,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables
0,1010140020171,1,1,0,1,0,1,1,1,0,0,1,1
1,1010140020284,1,1,0,0,0,0,0,1,0,0,0,1
2,1010140020297,1,1,0,0,0,1,1,1,1,0,1,1
3,1010140020409,1,1,0,0,0,1,0,1,0,0,1,1
4,1010140020471,1,1,0,0,0,0,0,1,1,0,1,1


In [371]:
new_data.describe()

food_group,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables
count,3.265000e+03,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000,3265.000000
mean,1.654744e+13,0.942726,0.972129,0.185605,0.646248,0.535681,0.496172,0.311179,0.744564,0.874426,0.755590,0.749464,0.937825
std,1.599970e+13,0.232401,0.164629,0.388847,0.478207,0.498802,0.500062,0.463047,0.436173,0.331420,0.429803,0.433388,0.241509
min,1.010140e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.010212e+12,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
50%,1.101002e+13,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.802031e+13,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,5.502018e+13,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [372]:
new_data.columns

Index(['hhid', 'cereals', 'condiments', 'eggs', 'fish_seafoods', 'fruits',
       'meat', 'milk', 'oil_fats', 'pulses_nuts', 'roots_tubers', 'sugar',
       'vegetables'],
      dtype='object', name='food_group')

In [373]:
#calculate the HDDS 
new_data['hdds']  = (
    new_data['cereals'] + 
    new_data['pulses_nuts'] +
    new_data['eggs'] +
    new_data['fish_seafoods'] +
    new_data['meat'] + 
    new_data['milk'] + 
    new_data['roots_tubers'] + 
    new_data['vegetables'] + 
    new_data['sugar'] + 
    new_data['fruits'] +
    new_data['oil_fats'] + 
    new_data['condiments']
)
new_data.head()

food_group,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables,hdds
0,1010140020171,1,1,0,1,0,1,1,1,0,0,1,1,8
1,1010140020284,1,1,0,0,0,0,0,1,0,0,0,1,4
2,1010140020297,1,1,0,0,0,1,1,1,1,0,1,1,8
3,1010140020409,1,1,0,0,0,1,0,1,0,0,1,1,6
4,1010140020471,1,1,0,0,0,0,0,1,1,0,1,1,6


In [374]:
#save the file may be useful later
new_data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_hdds.csv', index=False)

### Combine The Food Indicators and Location

In [375]:
#read the file containing the location of the household 
location =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_region.csv', header=0,delimiter=',')
location.head()

,hhid,year,region,district,district_name,ward,locality
0,1010140020171,2008,DODOMA,1,KONDOA,14,Rural
1,1010140020284,2008,DODOMA,1,KONDOA,14,Rural
2,1010140020297,2008,DODOMA,1,KONDOA,14,Rural
3,1010140020409,2008,DODOMA,1,KONDOA,14,Rural
4,1010140020471,2008,DODOMA,1,KONDOA,14,Rural


In [376]:
location.shape

(3264, 7)

In [377]:
#read the files containing household hdds
hdds =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_hdds.csv', header=0,delimiter=',')
hdds.head()

,hhid,cereals,condiments,eggs,fish_seafoods,fruits,meat,milk,oil_fats,pulses_nuts,roots_tubers,sugar,vegetables,hdds
0,1010140020171,1,1,0,1,0,1,1,1,0,0,1,1,8
1,1010140020284,1,1,0,0,0,0,0,1,0,0,0,1,4
2,1010140020297,1,1,0,0,0,1,1,1,1,0,1,1,8
3,1010140020409,1,1,0,0,0,1,0,1,0,0,1,1,6
4,1010140020471,1,1,0,0,0,0,0,1,1,0,1,1,6


In [378]:
#read the files containing household hdds
fcs =  read_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_with_fcs.csv', header=0,delimiter=',')
fcs.head()

,hhid,animal_protein,cereals_tubers,condiments,dairy_products,fruits,oil,pulses_nuts,sugar,vegetables_leaves,fcs
0,1010140020171,2,1,2,1,0,1,0,1,2,17.0
1,1010140020284,0,1,1,0,0,1,0,0,2,4.5
2,1010140020297,3,2,2,1,0,1,1,1,2,26.0
3,1010140020409,1,2,2,0,0,1,0,1,2,11.0
4,1010140020471,0,3,2,0,0,1,1,1,2,12.0


In [379]:
#merging the location,fcs and hdds based on household id
data = pd.merge(location, fcs[['hhid', 'fcs']], on='hhid', how='left')
data = pd.merge(data, hdds[['hhid', 'hdds']], on='hhid', how='left')
data.head(20)

,hhid,year,region,district,district_name,ward,locality,fcs,hdds
0,1010140020171,2008,DODOMA,1,KONDOA,14,Rural,17.0,8
1,1010140020284,2008,DODOMA,1,KONDOA,14,Rural,4.5,4
2,1010140020297,2008,DODOMA,1,KONDOA,14,Rural,26.0,8
3,1010140020409,2008,DODOMA,1,KONDOA,14,Rural,11.0,6
4,1010140020471,2008,DODOMA,1,KONDOA,14,Rural,12.0,6
5,1010140020551,2008,DODOMA,1,KONDOA,14,Rural,18.0,8
6,1010140020761,2008,DODOMA,1,KONDOA,14,Rural,8.5,5
7,1010140020762,2008,DODOMA,1,KONDOA,14,Rural,11.0,6
8,1020030030004,2009,DODOMA,2,MPWAPWA,3,Rural,30.0,10
9,1020030030022,2009,DODOMA,2,MPWAPWA,3,Rural,3.0,3


In [380]:
data.shape

(3264, 9)

In [381]:
#save the final dataset 
data.to_csv('Tanzania/2008_2009/tanzania_2008_2009_preprosessed_data/tanzania_2008_2009_final.csv', index=False)

In [382]:
data.head()

,hhid,year,region,district,district_name,ward,locality,fcs,hdds
0,1010140020171,2008,DODOMA,1,KONDOA,14,Rural,17.0,8
1,1010140020284,2008,DODOMA,1,KONDOA,14,Rural,4.5,4
2,1010140020297,2008,DODOMA,1,KONDOA,14,Rural,26.0,8
3,1010140020409,2008,DODOMA,1,KONDOA,14,Rural,11.0,6
4,1010140020471,2008,DODOMA,1,KONDOA,14,Rural,12.0,6
